In [ ]:
%matplotlib inline
%config InlineBackend.close_figures=False 
import ipywidgets as widgets
from IPython.display import display,clear_output, Audio
from sklearn.metrics import confusion_matrix
from ipywidgets import HBox, VBox, Layout
import matplotlib.pyplot as plt
from datetime import datetime
from random import shuffle
import sounddevice as sd
import pandas as pd
import numpy as np
import wavio
import time
import os


# Import classes and init random indexes
results = {'index': [],
           'truth': [],
           'prediction': [],
           'issue': []}
df_summary = pd.read_csv('kitchen20.csv')
rand_idxs = list(range(len(df_summary)))
shuffle(rand_idxs)
name = None

# Read audio fuction
def play(idx):
    row = df_summary.loc[idx]
    audioRate = 44100
    sound = wavio.read(row.path).data.T[0]
    sd.play(sound, audioRate)
    
    
def save(df_results, name):
    if not os.path.exists('./results'):
        os.makedirs('./results')
    df_results.to_csv('./results/{}_{:%y%m%d_%T}.csv'.format(
        name, datetime.now()))
    print('Saved')

    
def plot_confusion_matrix(y_true, y_pred, classes,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    fig, ax = plt.subplots(figsize=(10,9))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


#######################
# Instantiate the UI

### Class buttons
class_buttons = widgets.ToggleButtons(
    options=['blender', 'boiling-water', 'book', 'chopping', 'clean-dishes',
             'cupboard', 'cutlery', 'dishwasher', 'drawer', 'eat', 'fridge',
             'frying-pan', 'juicer', 'microwave', 'peel', 'plates', 'stove-burner',
             'stove-fan', 'sweep', 'water-flowing'],
    description='Class:')

### Play button    
play_button = widgets.Button(
    description='replay audio {}  -  1/60'.format(rand_idxs[-1]),
    layout=Layout(width='auto'))

@play_button.on_click
def play_audio(b):
    play(rand_idxs[-1])

### Issue text
issue_text = widgets.Text(
    description='Report issue:',
    value='',
    continus_update=True,
    disabled=False)

### Answer button
answer_button = widgets.Button(
    description='',
    layout=Layout(width='auto'))

### Next button
next_button = widgets.Button(
    description='Next',
    layout=Layout(width='auto'))

@next_button.on_click
def next_audio(b):
    # Record prediction
    played_idx = rand_idxs[-1]
    results['index'].append(played_idx)
    results['truth'].append(df_summary.category[played_idx])
    results['prediction'].append(class_buttons.value)
    results['issue'].append(issue_text.value)
    
    # Show answer
    answer_button.layout.visibility = 'visible'
    answer_button.description = df_summary.category[played_idx]
    if df_summary.category[played_idx] == class_buttons.value:
        answer_button.style.button_color = 'lightgreen'
    else:
        answer_button.style.button_color = 'red'
    time.sleep(1)
    answer_button.layout.visibility = 'hidden'

    # Change current index
    rand_idxs.pop()
    play(rand_idxs[-1])
    counter = len(df_summary) - len(rand_idxs) + 1
    
    # Update view
    if counter < 60:
        play_button.description='replay audio {}  -  {}/60'.format(
            rand_idxs[-1], counter)
        next_button.layout.visibility = 'hidden'
        issue_text.value = ''
        time.sleep(4)
        next_button.layout.visibility = 'visible'
    else:  # End game
        tab.layout.visibility = 'hidden'
        next_button.layout.visibility = 'hidden'
        df_results = pd.DataFrame(results)
        save(df_results, name)
        classes = list(set(df_results.truth.append(df_results.prediction)))
        y_test = [classes.index(c) for c in df_results.truth]
        y_pred = [classes.index(c) for c in df_results.prediction]
        plot_confusion_matrix(y_test, y_pred, classes=classes)
        acc = (np.array(y_test) == np.array(y_pred)).mean() * 100
        print('Your accuracy is {}%'.format(acc))
        print('Thaaaaaank youuuuuu :D')
    
# Define visual box
tab = VBox(children=[play_button,
                     class_buttons,
                     issue_text,
                     next_button,
                     answer_button])

name = input('What is your name ?')
display(tab)
play(rand_idxs[-1])